# Landsat V2 - Extraction avec Buffer et Statistiques

## Améliorations par rapport à V1

**V1** : Extrait un **pixel unique** au point de mesure

**V2** : Extrait un **buffer de 200m** autour du point avec **statistiques** :

| Problème V1 | Solution V2 |
|-------------|-------------|
| Un pixel = bruit de mesure | Buffer = valeur plus stable |
| Pas d'info sur l'hétérogénéité | Écart-type = variabilité locale |
| Sensible aux erreurs GPS | Buffer = plus robuste |

## Nouvelles features

Pour chaque bande/indice, on calcule :

| Suffixe | Description | Utilité |
|---------|-------------|--------|
| (rien) | Moyenne dans le buffer | Valeur représentative |
| `_std` | Écart-type dans le buffer | Hétérogénéité spatiale |

**Exemple** : Pour NDVI, on aura `NDVI` (moyenne) et `NDVI_std` (variabilité)

---

## Installation et imports

In [1]:
!pip install -q odc-stac planetary-computer pystac-client


[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import pystac_client
import planetary_computer as pc
from odc.stac import stac_load
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import os

print("Imports OK!")

Imports OK!


In [3]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Bandes spectrales à extraire
ALL_BANDS = ["blue", "green", "red", "nir08", "swir16", "swir22"]

# Taille du buffer en degrés (~200m à cette latitude)
# 1 degré ≈ 111 km, donc 0.002 ≈ 220m
BUFFER_SIZE = 0.002

# Période de recherche
DATE_RANGE = "2011-01-01/2015-12-31"

# Seuil de couverture nuageuse
MAX_CLOUD_COVER = 20

# Nombre de workers parallèles
N_WORKERS = 6

# Chemins
WATER_QUALITY_FILE = "../data/raw/water_quality_training_dataset.csv"
SUBMISSION_FILE = "../data/raw/submission_template.csv"
OUTPUT_DIR = "../data/processed"

print(f"Configuration:")
print(f"  Buffer: {BUFFER_SIZE} degrés (~{BUFFER_SIZE * 111000:.0f}m)")
print(f"  Seuil nuages: {MAX_CLOUD_COVER}%")
print(f"  Workers: {N_WORKERS}")

Configuration:
  Buffer: 0.002 degrés (~222m)
  Seuil nuages: 20%
  Workers: 6


---

## Fonctions d'extraction

In [4]:
def compute_indices(data):
    """
    Calcule les indices spectraux à partir des bandes.
    
    Paramètres:
        data : xarray.Dataset avec les bandes
    
    Retourne:
        dict avec les indices calculés (arrays 2D)
    """
    eps = 1e-10
    indices = {}
    
    nir = data['nir08'].values.astype(float)
    green = data['green'].values.astype(float)
    red = data['red'].values.astype(float)
    swir16 = data['swir16'].values.astype(float)
    
    # NDVI = (NIR - Red) / (NIR + Red)
    indices['NDVI'] = (nir - red) / (nir + red + eps)
    
    # NDWI = (Green - NIR) / (Green + NIR)
    indices['NDWI'] = (green - nir) / (green + nir + eps)
    
    # NDMI = (NIR - SWIR16) / (NIR + SWIR16)
    indices['NDMI'] = (nir - swir16) / (nir + swir16 + eps)
    
    # MNDWI = (Green - SWIR16) / (Green + SWIR16)
    indices['MNDWI'] = (green - swir16) / (green + swir16 + eps)
    
    return indices

In [5]:
def extract_landsat_with_buffer(df, buffer_size=BUFFER_SIZE, max_cloud_cover=MAX_CLOUD_COVER, 
                                 n_workers=N_WORKERS, save_every=50, 
                                 backup_path="../data/processed/landsat_buffer_backup.csv"):
    """
    Extrait les valeurs Landsat avec buffer et statistiques.
    
    Pour chaque point :
    1. Crée un buffer carré autour du point
    2. Charge les pixels Landsat dans ce buffer
    3. Calcule moyenne et écart-type pour chaque bande/indice
    
    Retourne:
        DataFrame avec colonnes: bande, bande_std, indice, indice_std
    """
    df = df.copy().reset_index(drop=True)
    
    # Liste des features à extraire
    bands = ['blue', 'green', 'red', 'nir', 'swir16', 'swir22']
    indices = ['NDVI', 'NDWI', 'NDMI', 'MNDWI']
    
    # Colonnes de résultats (moyenne + std)
    result_cols = bands + [f'{b}_std' for b in bands] + indices + [f'{i}_std' for i in indices]
    results = {col: np.full(len(df), np.nan) for col in result_cols}
    
    completed_count = 0
    
    print(f"Extraction avec buffer pour {len(df)} points")
    print(f"  Buffer: ~{buffer_size * 111000:.0f}m | Nuages max: {max_cloud_cover}%")
    print(f"  Sauvegarde tous les {save_every} points")
    print("="*60)
    
    def extract_point(args):
        """Extrait les données pour un point avec buffer."""
        idx, lat, lon, sample_date = args
        
        try:
            if pd.isna(sample_date):
                return idx, {col: np.nan for col in result_cols}
            
            if isinstance(sample_date, str):
                sample_date = pd.to_datetime(sample_date, dayfirst=True, errors='coerce')
            if pd.isna(sample_date):
                return idx, {col: np.nan for col in result_cols}
            
            # Bounding box avec buffer
            bbox = [
                lon - buffer_size,
                lat - buffer_size,
                lon + buffer_size,
                lat + buffer_size
            ]
            
            # Connexion au catalogue
            catalog = pystac_client.Client.open(
                "https://planetarycomputer.microsoft.com/api/stac/v1",
                modifier=pc.sign_inplace,
            )
            
            # Recherche des scènes
            search = catalog.search(
                collections=["landsat-c2-l2"],
                bbox=bbox,
                datetime=DATE_RANGE,
                query={"eo:cloud_cover": {"lt": max_cloud_cover}},
            )
            items = list(search.item_collection())
            
            if not items:
                return idx, {col: np.nan for col in result_cols}
            
            # Sélectionner la scène la plus proche de la date
            sample_date_utc = sample_date.tz_localize("UTC") if sample_date.tzinfo is None else sample_date
            best_item = min(items, key=lambda x: abs(
                pd.to_datetime(x.properties["datetime"]).tz_convert("UTC") - sample_date_utc
            ))
            
            # Charger les données
            signed_item = pc.sign(best_item)
            data = stac_load([signed_item], bands=ALL_BANDS, bbox=bbox).isel(time=0)
            
            result = {}
            
            # Statistiques pour chaque bande
            band_mapping = {'nir08': 'nir'}  # Renommer nir08 -> nir
            
            for band in ALL_BANDS:
                band_data = data[band].values.astype(float)
                band_data = band_data[band_data > 0]  # Exclure les valeurs 0 (nodata)
                
                output_name = band_mapping.get(band, band)
                
                if len(band_data) > 0:
                    result[output_name] = float(np.nanmean(band_data))
                    result[f'{output_name}_std'] = float(np.nanstd(band_data))
                else:
                    result[output_name] = np.nan
                    result[f'{output_name}_std'] = np.nan
            
            # Calculer les indices et leurs stats
            computed_indices = compute_indices(data)
            
            for idx_name, idx_data in computed_indices.items():
                idx_flat = idx_data.flatten()
                idx_flat = idx_flat[~np.isnan(idx_flat)]  # Exclure NaN
                idx_flat = idx_flat[(idx_flat >= -1) & (idx_flat <= 1)]  # Valeurs valides
                
                if len(idx_flat) > 0:
                    result[idx_name] = float(np.nanmean(idx_flat))
                    result[f'{idx_name}_std'] = float(np.nanstd(idx_flat))
                else:
                    result[idx_name] = np.nan
                    result[f'{idx_name}_std'] = np.nan
            
            return idx, result
            
        except Exception as e:
            return idx, {col: np.nan for col in result_cols}
    
    # Préparer les arguments
    args_list = [
        (idx, df.loc[idx, 'Latitude'], df.loc[idx, 'Longitude'], df.loc[idx, 'Sample Date'])
        for idx in df.index
    ]
    
    # Extraction parallèle
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = {executor.submit(extract_point, args): args[0] for args in args_list}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Extraction"):
            try:
                idx, point_results = future.result(timeout=120)
                for col in result_cols:
                    if col in point_results:
                        results[col][idx] = point_results[col]
                
                completed_count += 1
                
                # Sauvegarde incrémentale
                if completed_count % save_every == 0:
                    backup_df = pd.DataFrame(results)
                    backup_df.to_csv(backup_path, index=False)
                    
            except Exception:
                pass
    
    # Sauvegarde finale
    final_df = pd.DataFrame(results)
    final_df.to_csv(backup_path, index=False)
    print(f"\nExtraction terminée! Sauvegarde: {backup_path}")
    
    return final_df

print("Fonction extract_landsat_with_buffer() définie!")

Fonction extract_landsat_with_buffer() définie!


---

## Extraction pour les données d'entraînement

⏱️ **Temps estimé** : ~3-4 heures pour ~9300 points

In [6]:
# Charger les données de qualité d'eau
water_quality = pd.read_csv(WATER_QUALITY_FILE)
print(f"Sites training : {len(water_quality)}")
display(water_quality.head())

Sites training : 9319


,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-28.760833,17.730278,02-01-2011,128.912,555.0,10.0
1,-26.861111,28.884722,03-01-2011,74.720,162.9,163.0
2,-26.450000,28.085833,03-01-2011,89.254,573.0,80.0
3,-27.671111,27.236944,03-01-2011,82.000,203.6,101.0
4,-27.356667,27.286389,03-01-2011,56.100,145.1,151.0


In [7]:
# =============================================================================
# EXTRACTION TRAINING AVEC BUFFER
# =============================================================================

training_features = extract_landsat_with_buffer(
    water_quality,
    buffer_size=BUFFER_SIZE,
    max_cloud_cover=MAX_CLOUD_COVER,
    n_workers=N_WORKERS,
    save_every=50,
    backup_path="../data/processed/landsat_training_buffer_backup.csv"
)

print(f"\nFeatures extraites : {len(training_features.columns)} colonnes")
print(training_features.columns.tolist())

Extraction avec buffer pour 9319 points
  Buffer: ~222m | Nuages max: 20%
  Sauvegarde tous les 50 points


Extraction: 100%|██████████| 9319/9319 [3:27:37<00:00,  1.34s/it]  



Extraction terminée! Sauvegarde: ../data/processed/landsat_training_buffer_backup.csv

Features extraites : 20 colonnes
['blue', 'green', 'red', 'nir', 'swir16', 'swir22', 'blue_std', 'green_std', 'red_std', 'nir_std', 'swir16_std', 'swir22_std', 'NDVI', 'NDWI', 'NDMI', 'MNDWI', 'NDVI_std', 'NDWI_std', 'NDMI_std', 'MNDWI_std']


In [8]:
# Créer le DataFrame final avec coordonnées
training_df = pd.DataFrame({
    'Latitude': water_quality['Latitude'].values,
    'Longitude': water_quality['Longitude'].values,
    'Sample Date': water_quality['Sample Date'].values,
})

# Ajouter les features Landsat
for col in training_features.columns:
    training_df[col] = training_features[col].values

# Sauvegarder
output_path = os.path.join(OUTPUT_DIR, 'landsat_features_training_v2.csv')
training_df.to_csv(output_path, index=False)

print(f"Fichier sauvegardé : {output_path}")
print(f"  - {len(training_df)} lignes")
print(f"  - {len(training_df.columns)} colonnes")

# Statistiques
print(f"\nTaux de complétion :")
for col in ['blue', 'nir', 'NDVI', 'NDVI_std']:
    if col in training_df.columns:
        taux = (1 - training_df[col].isna().mean()) * 100
        print(f"  {col}: {taux:.1f}%")

Fichier sauvegardé : ../data/processed\landsat_features_training_v2.csv
  - 9319 lignes
  - 23 colonnes

Taux de complétion :
  blue: 98.9%
  nir: 98.9%
  NDVI: 99.1%
  NDVI_std: 99.1%


---

## Extraction pour les données de validation

⏱️ **Temps estimé** : ~15-20 minutes pour ~200 points

In [9]:
# Charger le template de submission
submission = pd.read_csv(SUBMISSION_FILE)
print(f"Sites validation : {len(submission)}")

Sites validation : 200


In [10]:
# =============================================================================
# EXTRACTION VALIDATION AVEC BUFFER
# =============================================================================

validation_features = extract_landsat_with_buffer(
    submission,
    buffer_size=BUFFER_SIZE,
    max_cloud_cover=MAX_CLOUD_COVER,
    n_workers=N_WORKERS,
    save_every=20,
    backup_path="../data/processed/landsat_validation_buffer_backup.csv"
)

Extraction avec buffer pour 200 points
  Buffer: ~222m | Nuages max: 20%
  Sauvegarde tous les 20 points


Extraction: 100%|██████████| 200/200 [04:11<00:00,  1.26s/it]


Extraction terminée! Sauvegarde: ../data/processed/landsat_validation_buffer_backup.csv


In [11]:
# Créer le DataFrame final
validation_df = pd.DataFrame({
    'Latitude': submission['Latitude'].values,
    'Longitude': submission['Longitude'].values,
    'Sample Date': submission['Sample Date'].values,
})

for col in validation_features.columns:
    validation_df[col] = validation_features[col].values

# Sauvegarder
output_path = os.path.join(OUTPUT_DIR, 'landsat_features_validation_v2.csv')
validation_df.to_csv(output_path, index=False)

print(f"Fichier sauvegardé : {output_path}")
print(f"  - {len(validation_df)} lignes")
print(f"  - {len(validation_df.columns)} colonnes")

Fichier sauvegardé : ../data/processed\landsat_features_validation_v2.csv
  - 200 lignes
  - 23 colonnes


---

## Résumé des features créées

### Bandes spectrales (12 features)

| Bande | Moyenne | Écart-type |
|-------|---------|------------|
| blue | `blue` | `blue_std` |
| green | `green` | `green_std` |
| red | `red` | `red_std` |
| nir | `nir` | `nir_std` |
| swir16 | `swir16` | `swir16_std` |
| swir22 | `swir22` | `swir22_std` |

### Indices spectraux (8 features)

| Indice | Moyenne | Écart-type | Interprétation std |
|--------|---------|------------|--------------------|
| NDVI | `NDVI` | `NDVI_std` | Hétérogénéité végétation |
| NDWI | `NDWI` | `NDWI_std` | Variation présence eau |
| NDMI | `NDMI` | `NDMI_std` | Variation humidité |
| MNDWI | `MNDWI` | `MNDWI_std` | Variation eau libre |

### Total : 20 features Landsat

### Fichiers créés

| Fichier | Description |
|---------|-------------|
| landsat_features_training_v2.csv | Training avec buffer + stats |
| landsat_features_validation_v2.csv | Validation avec buffer + stats |

### Pourquoi l'écart-type est utile ?

- **std élevé** = zone hétérogène (mélange eau/terre, bord de rivière)
- **std faible** = zone homogène (grand plan d'eau, forêt dense)
- Peut aider à distinguer rivière (std élevé) vs lac (std faible)